In [ ]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from time import time

In [ ]:
def d1(St, K, T, t, r, q, sigma):
    return (np.log(St/K) + (r -q + sigma**2/2)*(T-t))/(sigma*np.sqrt(T - t))

def d2(St, K, T, t, r, q, sigma):
    return (np.log(St/K) + (r - q - sigma**2/2)*(T-t))/(sigma*np.sqrt(T - t))

def BSMcall(St, K, T, t, r, q, sigma):
    """
    St: Stock price at time t
    t: reference time
    T: time to maturity
    r: risk free rate
    q : dividend rate
    sigma: volatility
    """
    _d1 = sp.special.ndtr(d1(St, K, T, t, r, q, sigma))
    _d2 = sp.special.ndtr(d2(St, K, T, t, r, q, sigma))

    return St*np.exp(-q*(T-t))*_d1 - K*np.exp(-r*(T-t))*_d2

def BSMdigitalCall(St, K, T, t, r, q, sigma):
    return np.exp(-r*(T-t))*sp.special.ndtr(d2(St, K, T, t, r, q, sigma))

def BSMput(St, K, T, t, r, q, sigma):
    _d1 = sp.special.ndtr(-d1(St, K, T, t, r, q, sigma))
    _d2 = sp.special.ndtr(-d2(St, K, T, t, r, q, sigma))

    return K*np.exp(-r*(T-t))*_d2 - St*np.exp(-q*(T-t))*_d1

def BSMdigitalPut(St, K, T, t, r, q, sigma):
    return np.exp(-r*(T-t))*sp.special.ndtr(-d2(St, K, T, t, r, q, sigma))

def BSMforwardPrice(St, K, T, t, r, q):
    return St*np.exp(-q*(T-t)) - K*np.exp(-r*(T-t))

def BSMzeroCouponBondPrice(T, t, r):
    return np.exp(-r*(T-t))

Forward rates are quoted as simple rates

e.g. If someone loan $\$5$ with annualized interest 5% over 3 year period, he is expected to pay back the $\$5$ + the 5% of $\$5$ times the accrual period. That is,

$ 5 + \frac{5}{100} \cdot 5 \cdot 3 = 5.75$

This is how you always find forward rates quoted in the market. Continuous compounding rates are not quoted!

$f = \frac{P(T)^{-1}-1}{T}$

PVing cashflows

$P(T_1) -(1+f_{12}(T_2-T_1))P(T_2) = 0$

$f_{12} = \frac{\frac{P(T_1)}{P(T_2}-1}{T_2-T_1}$

$f_{12}$ is the fair forward rate between the 2 specified dates

$\frac{P(T_2)}{P(T_1)} = P(T_1,T_2) = (1+f_{12}(T_2-T_1))^{-1}$

## **Forward Rate Agreements**

The value of a forward rate agreement is simply

$(f-K)(T_2-T_1)P(T_2)$, where $K$ is the strike of the FRA and $f$ is the forward rate between the two dates at the start of the agreeement

## **Caplets**

An option on an FRA is called a **Caplet** or **Floorlet** depending on which side we bet

The payoff of a caplet is

$ (f-K)_{+}(T_2-T_1)P(T_2) $

Payoff of a floorlet is

$ (K-f)_{+}(T_2-T_1)P(T_2) $


Let $C$ denote the value of the caplet. Then
\begin{equation}
C_0 = P(0,T_2) E(\frac{C_{T_1}}{P(T_1,T_2)})
\end{equation}

, where $P(T_1,T_2)$ is the value at $T_1$ of a zero coupon bond expiring at $T_2$

$P(T_1,T_2) = \frac{P(T_1)}{P(T_2)}  = (1+f_{12}(T_2-T_1))$

Then

$P(0,T_2) = \frac{1}{P(T_2)} = (1+f_{02}T_2)$

or

$P(T_2) = (1+f_{02}T_2)^{-1} $

As a definition, remember, I can loan £$P(T_2)$ today to receive £1 at time $T_2$.

At the same time, by definition, I can loan £1 today to receive £$P(T_2)^{-1}$ or otherwise £$P(0,T_2)$ or otherwise £$ (1+f_{02}T_2)$ at $T_2$.

$C_0$ is the value of the caplet today $(t=0)$

\begin{equation}
C_0 = E((f-K)_{+})(T_2-T_1) P(0,T_2)
\end{equation}

Using Black's formula

\begin{equation}
C_0=P\left(0, T_2\right)\left(T_2-T_1\right) \mathrm{Q}\left(f_0, K, \sigma, T_1\right)
\end{equation}
,where
\begin{equation}
\mathrm{Q}\left(f_0, K, \sigma, T\right)=f_0 N\left(d_1\right)-K N\left(d_2\right)
\end{equation}

\begin{equation}
d_j=\frac{\log \left(\frac{f_0}{K}\right)+(-1)^{j-1} \frac{1}{2} \sigma^2 T_1}{\sigma \sqrt{T_1}}
\end{equation}

Mind you that we are using $P(T_2)$ as numeraire.



In [ ]:
def caplet(f0, K, T2, P2, T1, sigma):
    """
    f0: Stock price at time t
    K : strike
    T2: expiry date
    T1: start date of forward rate agreement
    sigma: volatility
    Ps : Discount curve
    """
    Q = BSMcall(St= f0, K = K, T = T1, t = 0, r = 0, q = 0, sigma = sigma)

    return Q*(T2-T1)*P2

def floorlet(f0, K, T2, P2, T1, sigma):
    """
    f0: Stock price at time t
    K : strike
    T2: expiry date
    T1: start date of forward rate agreement
    sigma: volatility
    Ps : Discount curve
    """
    Q = BSMput(St= f0, K = K, T = T1, t = 0, r = 0, q = 0, sigma = sigma)

    return Q*(T2-T1)*P2


In [ ]:
def computeAnnuity(P,taus):
    """
    Computes Annuity using the discount curve
    P : Discount curve
    taus : time steps
    """
    annuity = np.sum(P[1:]*taus)
    return annuity
def computeSwapRates(P,taus):
    """
    Computes Swap Rates using the discount curve
    P : Discount curve
    taus : time steps
    """
    annuity = computeAnnuity(P,taus)
    for i in range(1,N_steps):
        X[i-1] = (P[0] - P[i])/annuity

    return X


In [ ]:
# Now we have the Swap Rates and P0
# Goal to find out P(t_j)
def computeAnnuityFromSwapRates(SR, taus, P0):
    """
    Computes Annuity using the swap rates
    SR : Swap rates
    taus : time steps
    P0 : Present Value at t0
    """
    return P0*np.sum(taus)/(1+np.sum(taus*SR))

def computePs(SR,taus,P0):
    """
    Computes Discount curve using the swap rates
    SR : Swap rates
    taus : time steps
    P0 : Present Value at t0
    """
    annuity = computeAnnuityFromSwapRates(SR,taus,P0)
    return np.concatenate([np.array([P0]),P0 - annuity*SR])

In [ ]:
N_steps = 10

In [ ]:
# Generate discount curve
P = 0.5+np.random.uniform(0,1,N_steps)/10

times = np.linspace(0,1,N_steps)
taus = np.diff(times)
X = np.zeros(N_steps-1)

# Use Discount curve to compute Swap Rate and Annuity
X = computeSwapRates(P,taus)
B = computeAnnuity(P,taus)

# Use Swap Rates to compute Discount curve and Annuity
SRB = computeAnnuityFromSwapRates(X, taus, P[0])
SRP = computePs(X, taus, P[0])

# Check whether they're equal to one another
print(f'Annuities mismmatch is {(SRB-B)*100/B :.3f} %')
print(f'Discount curve match {SRP == P}')

Annuities mismmatch is 0.000 %
Discount curve match [ True  True  True  True  True  True  True  True  True  True]


## **Swaps**

With swaps, you are agreeing to pay floating interest whilst being paid the fixed interest on a notional

$X$ is the pre agreed fixed swap rate

The fixed interest payments are PVed to

$\Sigma_{j=0}^{n-1} X \tau_j P(T_{j+1}) $

Floating leg cashflows are PVed to

$ \Sigma_{j=0}^{n-1} f_j \tau_j P(T_{j+1}) $

Using the definition of $f_j\tau_jP(T_{j+1}) = P(T_j)-P(T_{j+1})$

So then these 2 cashflows are equal

$ X \Sigma_{j=0}^{n-1}\tau_j P(T_{j+1})  = \Sigma_{j=0}^{n-1} f_j \tau_j P(T_{j+1}) $

The annuity $B$ is defined to be $B = \Sigma_{j=0}^{n-1}\tau_j P(T_{j+1}) $

Thus

$ X =  \frac{\Sigma_{j=0}^{n-1} f_j \tau_j P(T_{j+1})}{B} $

or

$X = \frac{P_0 - P_N}{B}$


## **Swaptions**

An option to have the right to pay the fixed rate on a swap is called a payer's swaption. The right to pay the floating rate is called a receiver's swaption

If the strike of a swap is $K$ then the payoff of a payer's swaption is

$(X-K)_+B$

For a receiver's swaption it is

$(K-X)_+B$


Remember


$$X = \frac{P_0 - P_N}{B}$$

Both numerator and denominator are tradeables, as they are linear combinations of zero coupon bonds. Thus $X$ is a martingale and has $0$ drift if $B$ is taken as numeraire.

Then taking $O$ to denote the value of a payer's swaption we have

\begin{equation}
O_o = B_0 E(\frac{O_T}{B_T})
\end{equation}

or

\begin{equation}
O_o = B_0  E((X-K)_+)
\end{equation}


Using Black's formula

\begin{equation}
O_0= B_0  \mathrm{Q}\left(X_0, K, \sigma, T_1\right)
\end{equation}
,where
\begin{equation}
\mathrm{Q}\left(X_0, K, \sigma, T\right)=X_0 N\left(d_1\right)-K N\left(d_2\right)
\end{equation}

\begin{equation}
d_j=\frac{\log \left(\frac{X_0}{K}\right)+(-1)^{j-1} \frac{1}{2} \sigma^2 T_1}{\sigma \sqrt{T_1}}
\end{equation}

Mind you that we are using the annuity $B$ as numeraire.



### Implementing a Black Scholes analytical swaption

We assume we are given:
- Annuity
- Swap rate
- Strike
- Volatility
- Expiry

In [ ]:
def BSMPayerSwaption(X0, K, T2, T1, B, sigma):
    """
    BSM Swaption implementation

    X0 : initial swap rate
    K : strike
    T1: start of swap rate contract
    T2 : expiry
    B : Annuity
    sigma : volatility
    """
    Q = BSMcall(X0, K, T1, 0, 0, 0, sigma)
    return B*Q

In [ ]:
(BSMPayerSwaption(X0 = .045, K = .04, T2 = 2, T1 =1, B = 1, sigma =.2)*100)

np.float64(0.6453310427045815)

In [ ]:
def BSMPayerSwaption(X0, K, T, sigma):
    """
    BSM Payer Swaption implementation

    X0 : initial swap rate
    K : strike
    T : maturity
    sigma : volatility
    """

    return BSMcall(X0, K, T, 0, 0, 0, sigma)

def BSMReceiverSwaption(X0, K, T, sigma):
    """
    BSM Receiver Swaption implementation

    X0 : initial swap rate
    K : strike
    T : maturity
    sigma : volatility
    """

    return BSMput(X0, K, T, 0, 0, 0, sigma)

In [ ]:
BSMPayerSwaption(X0 = .045, K = .04, T = 1, sigma =.2)*100

np.float64(0.6453310427045815)

In [ ]:
BSMReceiverSwaption(X0 = .045, K = .04, T = 1, sigma =.2)*100

np.float64(0.1453310427045813)